In [4]:
import pandas as pd
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [5]:
load_dotenv(verbose=True)

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials())

In [6]:
def get_all_track_ids_from_albums_referenced_in_playlist(playlist_id: str) -> list:
    playlist_to_album_ids = lambda results : [item['track']['album']['id'] for item in results['items']]

    album_ids = []
    results = sp.playlist_items(playlist_id=playlist_id)
    album_ids.extend(playlist_to_album_ids(results))
    while results['next']:
        results = sp.next(results)
        album_ids.extend(playlist_to_album_ids(results))

    album_ids = list(set(album_ids))

    print(f'{len(album_ids)} albums referenced in playlist')

    # TODO: use an endpoint for multiple albums here?
    track_ids = []
    for album_id in album_ids:
        track_ids.extend([item['id'] for item in sp.album_tracks(album_id=album_id)['items']])

    print(f'{len(track_ids)} total tracks in those albums')

    return track_ids

In [2]:
def get_tracks_info_and_features(song_ids: list) -> pd.DataFrame:

    # loop through song_ids in chunks of 50 and call both .tracks and .audio_features
    # create rows with the columns: id, title, artist, album, genre, and then all of the relevant audio features
    # (this project will use genre as the prediction target, but it could be useful to save other things b/c why not)

    pass

NameError: name 'pd' is not defined

In [9]:
SMALL_TESTER_PLAYLIST = '37i9dQZF1DWXT8uSSn6PRy'
BIG_BOI_PLAYLIST = '37i9dQZF1DWTmvXBN4DgpA'

track_ids = get_all_track_ids_from_albums_referenced_in_playlist(SMALL_TESTER_PLAYLIST)
track_ids
# tracks_df = get_tracks_info_and_features(track_ids)

75 albums referenced in playlist
352 total tracks in those albums


['6kxaaIeowajN7w21PfMLbu',
 '7mFj0LlWtEJaEigguaWqYh',
 '0TyUOnU4H4GLqOcrH0auc8',
 '7GSwmKxanoWEdpurOCldDe',
 '6HMtHNpW6YPi1hrw9tgF8P',
 '1iV2uSjP8BfHTXVLblBOxI',
 '2rmwqU7yzTvzkiaRV53DpT',
 '3WjCwQWPM8ggvyEwUPFBZ5',
 '3m7IYU7ySuFMwdm7OCShQN',
 '7Bzgs1crJViC1NEliHSp8O',
 '5uhaFPXTIG06S9Hb2c5tsT',
 '0I6PjTwqXQlsJYwwBMP1sQ',
 '7IoLdpqB5igE13t12SKV8g',
 '52GgzMLY2W5VKqxO7nkQAN',
 '2GpCh34HpurCeyeHvSe8To',
 '6xGruZOHLs39ZbVccQTuPZ',
 '0xzI1KAr0Yd9tv8jlIk3sn',
 '079mntueS2KWjxVqOHjG70',
 '0hh6usshPzFf6j0lGkhMs3',
 '3RhyHYnYxuGnP8njFlNxHq',
 '6uNv4npQ8FZLsH7naE9GK4',
 '4ewmHD3aRBmLyTsbGreNLw',
 '4bt7eqP7mfWduRBKEpCAzM',
 '1bXm6XU3SYSqQGLTRhUSfm',
 '3yoIliXhnorEdfTZ93W2YL',
 '7l9IqDtVWJurTvkQHq1BGh',
 '4UbItCyP9q7MvuVogbDCgC',
 '5XLgE4yoU9MyR5vkjNxpdF',
 '03zuGqxMwUFFmCwusq0WKE',
 '40yVW4Ywyxz8xkNcuLbP1N',
 '2RwpoqhsYOZSGsF69KqjwP',
 '4cacyP5c3PMlfnyjpg13xW',
 '2PDgArI0p7UkeYgXWuqpmh',
 '3LtpKP5abr2qqjunvjlX5i',
 '6f5ExP43esnvdKPddwKXJH',
 '3GdWfmQBiiJrDUvSZS1bGv',
 '1WCEAGGRD066z2Q89ObXTq',
 